In [2]:
import netCDF4 as nc
import numpy as np


In [3]:
BASE_DIR='/prj/wetlands_africa/jules/JASMIN/BVOCs/SITE_DATA/'

infile=BASE_DIR+'Isoprene_Flux_AuchencorthMoss_2015.csv'
outfile=BASE_DIR+'Isoprene_Flux_AuchencorthMoss_2015.nc'

#infile=BASE_DIR+'Isoprene_Flux_AliceHolt_2005.csv'
#outfile=BASE_DIR+'Isoprene_Flux_AliceHolt_2005.nc'

fill_value=-999.
out_time_units='seconds since 2000-01-01 00:00:00'


In [4]:
inlines=open(infile,'r').readlines()
headers=inlines.pop(0).split(',')

In [5]:
print(headers)
short_headers=['time','t','par','isoprene','isoprene_err','isoprene_mmr']
units=[out_time_units,'C','umol m-2 s-1','ug m-2 h-1','ug m-2 h-1','ug m-3']

['Date & Time [UTC]', 'DOY', 'Temperature [C]', 'PAR [umol m-2 s-1]', 'Isoprene Flux [ug m-2 h-1]', 'Isoprene Flux Random Error [ug m-2 h-1]', 'Isoprene Concentration [ug m-3]\n']


In [6]:
DataDict={ hdr:[] for hdr in short_headers }
for line in inlines:
    split=line[:-1].split(',')
    DataDict['time'].append(nc.datetime.strptime(split[0],'%d/%m/%Y %H:%M'))
    for hdr,val in zip(short_headers[1:],split[2:]):
        if val != '':
            DataDict[hdr].append(float(val))
        else:
            DataDict[hdr].append(fill_value)
    

In [7]:
outf=nc.Dataset(outfile,'w')
outf.createDimension('time',len(DataDict['time']))

for var,unit in zip(short_headers,units):
    outvar=outf.createVariable(var,'float32',('time'),fill_value=fill_value)
    outvar.units=unit
    
    if var == 'time':
        outvar[:] = nc.date2num(DataDict[var],units=out_time_units)
    else:
        outvar[:]=DataDict[var]

outf.title='Alice Holt Isoprene Flux Data'
outf.owner='Ben Langford'
outf.converted_by='Edward Comyn-Platt (edwcom@ceh.ac.uk)'
outf.close()
    

In [8]:
outf.close()

RuntimeError: NetCDF: Not a valid ID